In [1]:
#from __future__ import division, print_function
import sys
import os
#import json
import csv
import pandas as pd
import numpy as np
import pylab as pl
%pylab inline
#import seaborn
#from scipy.optimize import curve_fit, minimize
import pyproj
from pyproj import Proj, transform

from scipy.stats import mode

import geopandas as gp
from fiona.crs import from_epsg
#import subprocess
import shapely as shp
from datetime import datetime
from itertools import compress
from geopandas.tools import sjoin
from shapely.geometry import Point


Populating the interactive namespace from numpy and matplotlib


In [2]:
# write clean yearly raw data to files

cb_2013 = pd.read_csv('data_raw_cb/cb_year_2013.csv')
cb_2014 = pd.read_csv('data_raw_cb/cb_year_2014.csv')
cb_2015 = pd.read_csv('data_raw_cb/cb_year_2015.csv')
cb_2016 = pd.read_csv('data_raw_cb/cb_year_2016.csv')
cb_2017 = pd.read_csv('data_raw_cb/cb_year_2017.csv')

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cb_2013_pu = cb_2013[['start station id','starttime', 'end station id','tripduration']]
cb_2014_pu = cb_2014[['start station id','starttime', 'end station id','tripduration']]
cb_2015_pu = cb_2015[['start station id','starttime', 'end station id','tripduration']]
cb_2016_pu = cb_2016[['start station id','starttime', 'end station id','tripduration']]

cb_2013_do = cb_2013[['end station id','stoptime', 'start station id','tripduration']]
cb_2014_do = cb_2014[['end station id','stoptime', 'start station id','tripduration']]
cb_2015_do = cb_2015[['end station id','stoptime', 'start station id','tripduration']]
cb_2016_do = cb_2016[['end station id','stoptime', 'start station id','tripduration']]

In [4]:
cb_2013_do.rename(columns={'end station id':'station_id','stoptime':'do_time','start station id':'cb_do_num_trips','tripduration':'cb_do_trip_time'}, inplace=True)
cb_2014_do.rename(columns={'end station id':'station_id','stoptime':'do_time','start station id':'cb_do_num_trips','tripduration':'cb_do_trip_time'}, inplace=True)
cb_2015_do.rename(columns={'end station id':'station_id','stoptime':'do_time','start station id':'cb_do_num_trips','tripduration':'cb_do_trip_time'}, inplace=True)
cb_2016_do.rename(columns={'end station id':'station_id','stoptime':'do_time','start station id':'cb_do_num_trips','tripduration':'cb_do_trip_time'}, inplace=True)

/opt/rh/anaconda/root/lib/python2.7/site-packages/pandas/core/frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [5]:
cb_2013_pu.rename(columns={'start station id':'station_id','starttime':'pu_time','end station id':'cb_pu_num_trips','tripduration':'cb_pu_trip_time'}, inplace=True)
cb_2014_pu.rename(columns={'start station id':'station_id','starttime':'pu_time','end station id':'cb_pu_num_trips','tripduration':'cb_pu_trip_time'}, inplace=True)
cb_2015_pu.rename(columns={'start station id':'station_id','starttime':'pu_time','end station id':'cb_pu_num_trips','tripduration':'cb_pu_trip_time'}, inplace=True)
cb_2016_pu.rename(columns={'start station id':'station_id','starttime':'pu_time','end station id':'cb_pu_num_trips','tripduration':'cb_pu_trip_time'}, inplace=True)

In [6]:
cb_pu = cb_2013_pu.append(cb_2014_pu, ignore_index = True)
cb_pu = cb_pu.append(cb_2015_pu, ignore_index=True)
cb_pu = cb_pu.append(cb_2016_pu, ignore_index=True)
cb_pu.reset_index(drop=True,inplace = True)

cb_do = cb_2013_do.append(cb_2014_do, ignore_index=True)
cb_do = cb_do.append(cb_2015_do, ignore_index=True)
cb_do = cb_do.append(cb_2016_do, ignore_index=True)
cb_do.reset_index(drop=True,inplace = True)

In [7]:
cb_pu_3119 = cb_pu[cb_pu.station_id == 3119]
cb_pu_3119.reset_index(drop=True,inplace = True)

cb_do_3119 = cb_do[cb_do.station_id == 3119]
cb_do_3119.reset_index(drop=True,inplace = True)

In [8]:
cb_pu_3119.iloc[:,1:2] = pd.to_datetime(cb_pu_3119.pu_time).dt.date
cb_pu_3119.rename(columns={'pu_time':'pu_date'},inplace=True)
cb_pu_3119.iloc[:,2:3] = 1
cb_pu_3119.iloc[:,3:4] = cb_pu_3119.cb_pu_trip_time/60.0
cb_pu_3119 = cb_pu_3119[cb_pu_3119.pu_date <= pd.to_datetime('2016-06-30').to_datetime().date()]

cb_do_3119.iloc[:,1:2] = pd.to_datetime(cb_do_3119.do_time).dt.date
cb_do_3119.rename(columns={'do_time':'do_date'},inplace=True)
cb_do_3119.iloc[:,2:3] = 1
cb_do_3119.iloc[:,3:4] = cb_do_3119.cb_do_trip_time/60.0
cb_do_3119 = cb_do_3119[cb_do_3119.do_date <= pd.to_datetime('2016-06-30').to_datetime().date()]

/opt/rh/anaconda/root/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2885: FutureWarning: to_datetime is deprecated. Use self.to_pydatetime()
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
cb_pu_3119 = cb_pu_3119.groupby(by=['station_id','pu_date'],as_index=False).agg({'cb_pu_num_trips':"count", 'cb_pu_trip_time':"mean"})
cb_do_3119 = cb_do_3119.groupby(by=['station_id','do_date'],as_index=False).agg({'cb_do_num_trips':"count", 'cb_do_trip_time':"mean"})

In [10]:
yt_3119_pu = pd.read_csv('data_pre_yellow/yellow_3119_pu.csv').drop(['Unnamed: 0'],axis=1)
yt_3119_pu.reset_index(drop=True,inplace=True)
yt_3119_pu.rename(columns={'num_trips':'yt_pu_num_trips','trip_time':'yt_pu_trip_time','trip_distance':'yt_pu_trip_distance'},inplace=True)
yt_3119_pu.iloc[:,2:3] = pd.to_datetime(yt_3119_pu.pu_date).dt.date
#yt_3119_pu = yt_3119_pu.sort_values(by=['station_id','radius','pu_date'])[yt_3119_pu.pu_date <= pd.to_datetime('2016-06-30').to_datetime().date()]
yt_3119_pu = yt_3119_pu[yt_3119_pu.pu_date >= pd.to_datetime('2013-09-01').to_datetime().date()]
yt_3119_pu = yt_3119_pu[yt_3119_pu.pu_date <= pd.to_datetime('2016-06-30').to_datetime().date()]

yt_3119_do = pd.read_csv('data_pre_yellow/yellow_3119_do.csv').drop(['Unnamed: 0'],axis=1)
yt_3119_do.reset_index(drop=True,inplace=True)
yt_3119_do.rename(columns={'num_trips':'yt_do_num_trips','trip_time':'yt_do_trip_time','trip_distance':'yt_do_trip_distance'},inplace=True)
yt_3119_do.iloc[:,2:3] = pd.to_datetime(yt_3119_do.do_date).dt.date
#yt_3119_do = yt_3119_do.sort_values(by=['radius','station_id','do_date'])[yt_3119_do.do_date <= pd.to_datetime('2016-06-30').to_datetime().date()]
yt_3119_do = yt_3119_do[yt_3119_do.do_date >= pd.to_datetime('2013-09-01').to_datetime().date()]
yt_3119_do = yt_3119_do[yt_3119_do.do_date <= pd.to_datetime('2016-06-30').to_datetime().date()]


In [11]:
gt_3119_pu = pd.read_csv('data_pre_green/green_3119_pu.csv').drop(['Unnamed: 0'],axis=1)
gt_3119_pu.reset_index(drop=True,inplace=True)
gt_3119_pu.rename(columns={'num_trips':'gt_pu_num_trips','trip_time':'gt_pu_trip_time','trip_distance':'gt_pu_trip_distance'},inplace=True)
gt_3119_pu.iloc[:,2:3] = pd.to_datetime(gt_3119_pu.pu_date).dt.date
gt_3119_pu = gt_3119_pu[gt_3119_pu.pu_date >= pd.to_datetime('2013-09-01').to_datetime().date()]
gt_3119_pu = gt_3119_pu[gt_3119_pu.pu_date <= pd.to_datetime('2016-06-30').to_datetime().date()]

gt_3119_do = pd.read_csv('data_pre_green/green_3119_do.csv').drop(['Unnamed: 0'],axis=1)
gt_3119_do.reset_index(drop=True,inplace=True)
gt_3119_do.rename(columns={'num_trips':'gt_do_num_trips','trip_time':'gt_do_trip_time','trip_distance':'gt_do_trip_distance'},inplace=True)
gt_3119_do.iloc[:,2:3] = pd.to_datetime(gt_3119_do.do_date).dt.date
gt_3119_do = gt_3119_do[gt_3119_do.do_date >= pd.to_datetime('2013-09-01').to_datetime().date()]
gt_3119_do = gt_3119_do[gt_3119_do.do_date <= pd.to_datetime('2016-06-30').to_datetime().date()]

In [12]:
yt_3119_pu[yt_3119_pu.duplicated(subset=['station_id','radius','pu_date'],keep='last') == True]

,station_id,radius,pu_date,yt_pu_num_trips,yt_pu_trip_distance,yt_pu_trip_time


In [13]:
gt_3119_pu[gt_3119_pu.duplicated(subset=['station_id','radius','pu_date'],keep='last') == True]

,station_id,radius,pu_date,gt_pu_num_trips,gt_pu_trip_distance,gt_pu_trip_time


In [14]:
yt_3119_do[yt_3119_do.duplicated(subset=['station_id','do_date','radius'],keep='last') == True]

,station_id,radius,do_date,yt_do_num_trips,yt_do_trip_distance,yt_do_trip_time
61,3119,150,2013-11-01,1,3.200000,12.466667
182,3119,150,2014-03-01,1,2.700000,12.000000
244,3119,150,2014-05-01,1,4.310000,20.000000
337,3119,150,2014-08-01,1,4.150000,21.000000
703,3119,150,2015-08-01,1,7.440000,19.450000
977,3119,150,2016-05-01,1,3.100000,25.016667
1101,3119,300,2013-11-01,3,3.943333,18.561111
1222,3119,300,2014-03-01,3,6.883333,18.572222
1254,3119,300,2014-04-01,1,7.940000,13.000000
1285,3119,300,2014-05-01,2,6.840000,18.500000


In [15]:
gt_3119_do[gt_3119_do.duplicated(subset=['station_id','do_date','radius'],keep='last') == True]

,station_id,radius,do_date,gt_do_num_trips,gt_do_trip_distance,gt_do_trip_time
719,3119,150,2015-11-01,2,2.365,711.325000
1361,3119,300,2014-11-01,1,2.870,1433.083333
1423,3119,300,2015-01-01,1,7.200,24.666667
1483,3119,300,2015-03-01,1,4.900,27.450000
1698,3119,300,2015-10-01,1,0.570,1426.483333
1730,3119,300,2015-11-01,2,2.365,711.325000
1913,3119,300,2016-05-01,1,5.000,16.983333
2136,3119,450,2014-03-01,1,2.300,10.433333
2382,3119,450,2014-11-01,1,2.870,1433.083333
2413,3119,450,2014-12-01,1,4.350,18.466667


In [16]:
yt_3119_pu = yt_3119_pu.drop_duplicates(subset=['station_id','pu_date','radius'],keep='last')
gt_3119_pu = gt_3119_pu.drop_duplicates(subset=['station_id','pu_date','radius'],keep='last')

yt_3119_do = yt_3119_do.drop_duplicates(subset=['station_id','do_date','radius'],keep='last')
gt_3119_do = gt_3119_do.drop_duplicates(subset=['station_id','do_date','radius'],keep='last')

len(yt_3119_pu), len(gt_3119_pu), len(yt_3119_do), len(gt_3119_do)

(3102, 2961, 3101, 2982)

In [17]:
taxis_3119_pu = yt_3119_pu.merge(gt_3119_pu, how='outer', on=['station_id','radius','pu_date']).sort_values(by=['radius','pu_date'])
taxis_3119_do = yt_3119_do.merge(gt_3119_do, how='outer', on=['station_id','radius','do_date']).sort_values(by=['radius','do_date'])

In [18]:
taxis_3119_pu[taxis_3119_pu.duplicated(subset=['station_id','radius','pu_date'],keep='last') == True]

,station_id,radius,pu_date,yt_pu_num_trips,yt_pu_trip_distance,yt_pu_trip_time,gt_pu_num_trips,gt_pu_trip_distance,gt_pu_trip_time


In [19]:
taxis_3119_do[taxis_3119_do.duplicated(subset=['station_id','radius','do_date'],keep='last') == True]

,station_id,radius,do_date,yt_do_num_trips,yt_do_trip_distance,yt_do_trip_time,gt_do_num_trips,gt_do_trip_distance,gt_do_trip_time


In [20]:
len(taxis_3119_pu), len(taxis_3119_do)

(3102, 3102)

In [21]:
all_3119_pu = cb_pu_3119.merge(taxis_3119_pu, how='outer', on=['station_id','pu_date']).sort_values(by=['pu_date'])
all_3119_pu[all_3119_pu.duplicated(subset=['station_id','radius','pu_date'],keep='last') == True]

,station_id,pu_date,cb_pu_num_trips,cb_pu_trip_time,radius,yt_pu_num_trips,yt_pu_trip_distance,yt_pu_trip_time,gt_pu_num_trips,gt_pu_trip_distance,gt_pu_trip_time


In [22]:
all_3119_do = cb_do_3119.merge(taxis_3119_do, how='outer', on=['station_id','do_date']).sort_values(by=['do_date'])
all_3119_do[all_3119_do.duplicated(subset=['station_id','radius','do_date'],keep='last') == True]

,station_id,do_date,cb_do_trip_time,cb_do_num_trips,radius,yt_do_num_trips,yt_do_trip_distance,yt_do_trip_time,gt_do_num_trips,gt_do_trip_distance,gt_do_trip_time


In [23]:
len(all_3119_pu),len(all_3119_do)

(3102, 3102)

In [24]:
all_3119_do = all_3119_do.sort_values(by=['radius','do_date'])
all_3119_pu = all_3119_pu.sort_values(by=['radius','pu_date'])

In [25]:
yt_3119_pu.to_csv('data_out_cb/yt_3119_pu.csv')
gt_3119_pu.to_csv('data_out_cb/gt_3119_pu.csv')
yt_3119_do.to_csv('data_out_cb/yt_3119_do.csv')
gt_3119_do.to_csv('data_out_cb/gt_3119_do.csv')

In [26]:
all_3119_pu.to_csv('data_out_cb/all_3119_pu.csv')
all_3119_do.to_csv('data_out_cb/all_3119_do.csv')